In [1]:
import scgpt as scg
import torch as tc
import numpy as np
import pandas as pd
import scanpy as sc
from pathlib import Path
from sklearn.cluster import AgglomerativeClustering
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import ward, fcluster, dendrogram
from scipy.spatial.distance import pdist
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier



/home/pkeyl/.local/lib/python3.10/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/home/pkeyl/.local/lib/python3.10/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/home/pkeyl/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pkeyl/.local/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [2]:
#this is the right clustering script

In [3]:
embeddings = pd.read_parquet('./embeddings/VICREG_embedding_highinputdropout_5000epochs.parquet')#.lo
embeddings

,id,Pseudo,Tier_1,Tier_2,Tier_3,Tier_4,D0,D1,D2,D3,...,D54,D55,D56,D57,D58,D59,D60,D61,D62,D63
0,LC001_AAACAAGCAATAGAAGAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,-0.538435,-2.583414,-1.690637,-2.388549,...,3.626750,1.590095,0.929234,3.328943,2.065247,2.018108,-2.431058,2.865535,-0.677259,1.161320
1,LC001_AAACAAGCACCAGCTCAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,-0.795777,-0.060707,0.744286,-1.467339,...,-1.222340,0.659418,-0.452725,1.204869,-1.563786,-0.511233,0.167683,1.788337,-1.500667,1.641649
2,LC001_AAACAAGCACGGATACAACGGGAA-1,LC001,Tumor,Tumor,Tumor,Tumor,0.445498,0.995505,0.743160,-1.801469,...,0.059662,0.586445,1.651152,1.720395,-3.208083,-2.888699,2.066723,0.855179,-2.671105,-1.396526
3,LC001_AAACAAGCACTATGGAAACGGGAA-1,LC001,Tumor,Tumor,Tumor,Tumor,-0.637376,-0.889321,1.560297,-1.658672,...,-0.139208,-0.503284,5.617406,1.035638,-1.597914,3.544302,1.640252,-0.242066,-0.080449,3.440592
4,LC001_AAACAAGCATCACAGAAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,0.505654,2.079722,-0.103550,-0.826108,...,-3.014869,3.191186,-0.665614,4.924458,-1.494679,0.809119,2.162076,1.883955,-0.307100,2.552084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009020,LC402_TTTGGCGGTTTGAGCTATTCGGTT-1,LC402,Tumor,Tumor,Tumor,Tumor,-2.589694,-1.820872,1.148017,-2.563401,...,1.090960,-1.433417,2.273527,0.104501,-3.068588,1.093708,0.057958,2.894730,-0.717038,2.134637
2009021,LC402_TTTGTGAGTACCTCAGATTCGGTT-1,LC402,Epithelial cells,Ciliated cells,Ciliated cells,Ciliated cells,1.079565,1.236098,-1.310050,1.704180,...,-4.572837,7.536171,3.336499,0.385374,-8.509260,6.681077,-5.334517,-0.249203,-2.462911,0.705485
2009022,LC402_TTTGTGAGTAGGTGACATTCGGTT-1,LC402,Immune cells,Macrophages,"Macrophages, anti-inflammatory","Macrophages, anti-inflammatory",0.426734,4.510794,2.569683,-3.335322,...,1.637536,4.767130,3.830027,0.149011,0.629688,4.157779,-1.807609,3.450658,1.726974,0.914551
2009023,LC402_TTTGTGAGTGTCCTTCATTCGGTT-1,LC402,Mixed,Mixed,Mixed,Mixed,5.491659,4.062449,2.106006,-3.025659,...,0.978291,5.490396,4.390152,4.166881,1.553916,1.674141,3.930813,3.022021,-6.860493,1.130263


In [12]:
tc.manual_seed(0)
all_ids = tc.randperm(embeddings.shape[0]).numpy()
x_ids = all_ids[:50000]
y_ids = all_ids[:]
X = embeddings.iloc[x_ids, 6:]
Y = embeddings.iloc[:, 6:]

In [13]:
dist_mat = pdist(X)
Z = ward(dist_mat)


In [20]:
X_clusters = fcluster(Z, t=84.9, criterion='distance') 
len(set(X_clusters))

500

In [21]:
neigh = KNeighborsClassifier(n_neighbors=1)
#neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, X_clusters)

/home/pkeyl/.local/lib/python3.10/site-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


KNeighborsClassifier(n_neighbors=3)

In [22]:
Y_clusters = neigh.predict(Y)
Y_clusters

/home/pkeyl/.local/lib/python3.10/site-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


array([159, 441, 408, ..., 166, 183, 498], dtype=int32)

In [23]:
new_data_frame = embeddings.iloc[:,:6]
new_data_frame['cluster'] = Y_clusters
new_data_frame

,id,Pseudo,Tier_1,Tier_2,Tier_3,Tier_4,cluster
0,LC001_AAACAAGCAATAGAAGAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,159
1,LC001_AAACAAGCACCAGCTCAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,441
2,LC001_AAACAAGCACGGATACAACGGGAA-1,LC001,Tumor,Tumor,Tumor,Tumor,408
3,LC001_AAACAAGCACTATGGAAACGGGAA-1,LC001,Tumor,Tumor,Tumor,Tumor,393
4,LC001_AAACAAGCATCACAGAAACGGGAA-1,LC001,Mixed,Mixed,Mixed,Mixed,215
...,...,...,...,...,...,...,...
2009020,LC402_TTTGGCGGTTTGAGCTATTCGGTT-1,LC402,Tumor,Tumor,Tumor,Tumor,317
2009021,LC402_TTTGTGAGTACCTCAGATTCGGTT-1,LC402,Epithelial cells,Ciliated cells,Ciliated cells,Ciliated cells,81
2009022,LC402_TTTGTGAGTAGGTGACATTCGGTT-1,LC402,Immune cells,Macrophages,"Macrophages, anti-inflammatory","Macrophages, anti-inflammatory",166
2009023,LC402_TTTGTGAGTGTCCTTCATTCGGTT-1,LC402,Mixed,Mixed,Mixed,Mixed,183


In [24]:
#new_data_frame.to_parquet('./clusterings/all_trained_batchnorm_5600epochs_512batchsize.parquet')
new_data_frame.to_parquet('./clusterings/VICREG_clustering_highinputdropout_5000epochs.parquet')